In [1]:
import tweepy
import pandas as pd
import requests
import time

In [2]:
def process_tweet(tweet):
    mentions = []
    annotations = []
    context_annotations = []
    urls = []
    hashtags = []
    retweeting = "$"
    time_str = tweet.created_at
    if tweet.entities:
        keys = tweet.entities.keys()
    else:
        keys = []
    if tweet.text[:4] == "RT @":
        retweeting = (tweet.text.split("@")[1].split(":")[0])
    if 'annotations' in keys:
        for item in tweet.entities['annotations']:
            annotations.append(item['type'] + ":" + item['normalized_text'])
    if 'mentions' in keys:
        for item in tweet.entities['mentions']:
            mentions.append(item['username'])
    if 'urls' in keys:
        for item in tweet.entities['urls']:
            urls.append(item['expanded_url'])
    for item in tweet.context_annotations:
        if 'description' in item['entity'].keys():
            context_annotations.append(item['domain']['name'] + "," + item['entity']['name'] + "," + item['entity']['description'])
        else:
            context_annotations.append(item['domain']['name'] + "," + item['entity']['name'])
    if 'hashtags' in keys:
        for item in tweet.entities['hashtags']:
            hashtags.append(item['tag'])
    return str(time_str), str(mentions), str(annotations), str(context_annotations), str(urls), str(hashtags), retweeting

In [4]:
congdf = pd.read_csv("CongressAccounts.csv")
congdf.head()

,First Name,Last Name,Handle,State,Body,District,Party
0,Alma,Adams,RepAdams,NC,House,12.0,D
1,Robert,Aderholt,Robert_Aderholt,AL,House,4.0,R
2,Pete,Aguilar,RepPeteAguilar,CA,House,31.0,D
3,Rick,Allen,RepRickAllen,GA,House,12.0,R
4,Colin,Allred,RepColinAllred,TX,House,32.0,D


In [5]:
tweetsdf = pd.DataFrame({
    'Name': [],
    'Twitter Handle':[],
    'Time':[],
    'Content':[],
    'Retweeting':[],
    'Hashtags':[],
    'Mentions':[],
    'Annotations':[],
    'URLs':[],
    'Context Annotations':[]
})

In [6]:
num_tweets = len(tweetsdf['Name'])
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAAFJiYgEAAAAA8MDdMjXr%2BTe14hDs9gUFqcH9%2BQc%3DyFU2mifWLIH7JOZUCYY0KB9V5LlldlxBoy10UsAv5lboPEgRpz')

In [8]:
for i in range(0, len(congdf['First Name'])):
    if congdf['Handle'][i] != "None?":
        name = congdf['First Name'][i] + " " + congdf['Last Name'][i]
        print(name, len(tweetsdf['Name']))
        handle = congdf['Handle'][i]
        query = 'from:' + handle
        #try:
        tweets = client.search_recent_tweets(query=query, tweet_fields=['context_annotations', 'entities', 'lang', 'created_at'], max_results = 100)
        if tweets.data:
            print("Found",len(tweets.data),"tweets")
            for i in range(len(tweets.data)):
                tweet = tweets.data[i]
                #print(tweet.text)
                time_str, mentions, annotations, context_annotations, urls, hashtags, retweeting = process_tweet(tweet)
                tweetsdf.loc[num_tweets] = [name, handle, time_str, tweet.text, retweeting, hashtags, mentions, annotations, urls, context_annotations]
                num_tweets += 1
    #except Exception as exc:
        #    print(exc)
        time.sleep(4.5)

Alma Adams 0
Found 13 tweets
Robert Aderholt 13
Found 4 tweets
Pete Aguilar 17
Found 1 tweets
Rick Allen 18
Found 4 tweets
Colin Allred 22
Found 18 tweets
Mark Amodei 40
Found 3 tweets
Kelly Armstrong 43
Found 4 tweets
Jodey Arrington 47
Found 9 tweets
Jake Auchincloss 56
Found 5 tweets
Cynthia Axne 61
Found 5 tweets
Brian Babin 66
Found 10 tweets
Don Bacon 76
Found 31 tweets
James Baird 107
Found 8 tweets
Troy Balderson 115
Found 4 tweets
Jim Banks 119
Found 9 tweets
Andy Barr 128
Found 9 tweets
Nanette Barragán 137
Found 16 tweets
Karen Bass 153
Joyce Beatty 153
Found 8 tweets
Cliff Bentz 161
Found 8 tweets
Ami Bera 169
Found 8 tweets
Jack Bergman 177
Found 1 tweets
Donald Beyer 178
Found 68 tweets
Stephanie Bice 246
Found 8 tweets
Andy Biggs 254
Found 39 tweets
Gus Bilirakis 293
Found 20 tweets
Dan Bishop 313
Found 8 tweets
Sanford Bishop 321
Found 4 tweets
Earl Blumenauer 325
Lisa Blunt Rochester 325
Found 6 tweets
Lauren Boebert 331
Found 3 tweets
Suzanne Bonamici 334
Found 45 twe

In [9]:
tweetsdf.to_csv("Tweets11122022.csv")

In [37]:
tweetsdf['Context Annotations'][0].split("'")

['[',
 'Person,Alma Adams,US Representative Alma Adams (NC-12)',
 ', ',
 'Events [Entity Service],2022 United States midterm elections',
 ', ',
 'Politician,Alma Adams,US Representative Alma Adams (NC-12)',
 ', ',
 'Political Race,2022 United States midterm elections',
 ', ',
 'Unified Twitter Taxonomy,Politics,Politics',
 ', ',
 'Unified Twitter Taxonomy,Alma Adams,US Representative Alma Adams (NC-12)',
 ', ',
 'Unified Twitter Taxonomy,Political figures,Politician',
 ', ',
 'Unified Twitter Taxonomy,United States politics',
 ', ',
 'Unified Twitter Taxonomy,Political events',
 ', ',
 'Unified Twitter Taxonomy,United States political events',
 ', ',
 'Unified Twitter Taxonomy,2022 United States midterm elections',
 ']']

In [27]:
tweet = tweets.data[0]

In [28]:
tweet

<Tweet id=1591192185143439360 text=People in Wyoming place a high value of trust in their institutions. They bank where their grandfathers did. If the digital asset industry wants to have staying power, they need to build trust. My bill with @gillibrandny will build a framework to start building that trust.>

In [29]:
tweet.text

'People in Wyoming place a high value of trust in their institutions. They bank where their grandfathers did. If the digital asset industry wants to have staying power, they need to build trust. My bill with @gillibrandny will build a framework to start building that trust.'

In [30]:
for item in tweet.context_annotations:
    print(item['entity']['name'],item['domain']['name'])

Cynthia Lummis Person
Cynthia Lummis Politician
Politics Unified Twitter Taxonomy
Political figures Unified Twitter Taxonomy
Cynthia Lummis Unified Twitter Taxonomy
Digital goods & currency Unified Twitter Taxonomy
Wyoming States
Digital goods & currency Product Taxonomy
Kirsten Gillibrand Person
Kirsten Gillibrand Politician
Politics Unified Twitter Taxonomy
Kirsten Gillibrand Unified Twitter Taxonomy
Political figures Unified Twitter Taxonomy
